In [ ]:
!pip uninstall -y ultralytics

In [ ]:
!pip install ultralytics

In [1]:
import sys
import os
import gc
import torch
import torch.nn as nn
import yaml
from ultralytics import YOLO
from ultralytics.nn import tasks, modules

# ======================================================================================
# 1. [필수] CBAM 클래스 재정의 (이게 없으면 저장된 모델을 못 불러옵니다)
# ======================================================================================
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc1 = nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False)
        self.relu = nn.ReLU()
        self.fc2 = nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        avg_out = self.fc2(self.relu(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu(self.fc1(self.max_pool(x))))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        assert kernel_size in (3, 7), 'kernel size must be 3 or 7'
        padding = 3 if kernel_size == 7 else 1
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)

class CBAM_Universal(nn.Module):
    def __init__(self, *args, **kwargs): 
        super(CBAM_Universal, self).__init__()
        c1 = args[0]
        k = 7 
        self.channel_attention = ChannelAttention(c1)
        self.spatial_attention = SpatialAttention(k)
    def forward(self, x):
        out = self.channel_attention(x) * x
        out = self.spatial_attention(out) * out
        return out

# 시스템 등록 (Monkey Patching)
target_name = 'CBAM_Universal'
for module_list in [tasks, modules]:
    setattr(module_list, target_name, CBAM_Universal)
if 'ultralytics.nn.modules' in sys.modules:
    setattr(sys.modules['ultralytics.nn.modules'], target_name, CBAM_Universal)
print(f"✅ [시스템] '{target_name}' 등록 완료.")

# ======================================================================================
# 2. YAML 파일 생성 (설계도 복구)
# ======================================================================================
cbam_yaml = {
    'nc': 3, 
    'scales': { 'm': [0.67, 0.75, 768] },
    'backbone': [
        [-1, 1, 'Conv', [64, 3, 2]], [-1, 1, 'Conv', [128, 3, 2]], [-1, 3, 'C2f', [128, True]],
        [-1, 1, 'Conv', [256, 3, 2]], [-1, 6, 'C2f', [256, True]],
        [-1, 1, target_name, [192]], # 5
        [-1, 1, 'Conv', [512, 3, 2]], [-1, 6, 'C2f', [512, True]],
        [-1, 1, target_name, [384]], # 8
        [-1, 1, 'Conv', [1024, 3, 2]], [-1, 3, 'C2f', [1024, True]],
        [-1, 1, target_name, [576]], # 11
        [-1, 1, 'SPPF', [1024, 5]]   # 12
    ],
    'head': [
        [-1, 1, 'nn.Upsample', [None, 2, 'nearest']], [[-1, 8], 1, 'Concat', [1]], [-1, 3, 'C2f', [512]],
        [-1, 1, 'nn.Upsample', [None, 2, 'nearest']], [[-1, 5], 1, 'Concat', [1]], [-1, 3, 'C2f', [256]],
        [-1, 1, 'Conv', [256, 3, 2]], [[-1, 15], 1, 'Concat', [1]], [-1, 3, 'C2f', [512]],
        [-1, 1, 'Conv', [512, 3, 2]], [[-1, 11], 1, 'Concat', [1]], [-1, 3, 'C2f', [1024]],
        [[18, 21, 24], 1, 'Detect', ['nc']]
    ]
}
yaml_name = 'yolov8m_cbam_real_final.yaml'
with open(yaml_name, 'w') as f:
    yaml.dump(cbam_yaml, f, sort_keys=False)

# ======================================================================================
# 3. [수정됨] Class Loss 강화 실험 (배경 오인식 해결용)
# ======================================================================================
data_path = "C:/Users/hyun8/Desktop/Arakon/detection-base-6/data.yaml"
# 640 모델 경로 (본인 경로 확인 필수)
weight_640 = "runs/detect/tank_yolo_cbam_640_final/weights/best.pt"

print(f"\n🚀 [실험] Class Loss Tuning (배경 오인식 억제 실험)")

if not os.path.exists(weight_640):
    print(f"🚨 오류: 640 모델 파일({weight_640})을 찾을 수 없습니다.")
else:
    print(f"   ▶ {weight_640} 로드 후 학습 시작")
    
    # 메모리 청소
    gc.collect()
    torch.cuda.empty_cache()

    try:
        model = YOLO(yaml_name).load(weight_640)
        
        model.train(
            data=data_path, 
            epochs=100, 
            patience=20,
            imgsz=800,        
            batch=4,          
            
            # [핵심 수정] fl_gamma를 지우고 cls를 높입니다.
            # 이 설정은 "박스 위치"보다 "이게 진짜 탱크가 맞는지(분류)"를 더 꼼꼼하게 봅니다.
            cls=2.0,      # (기본값 0.5 -> 2.0으로 4배 강화)
            
            # 다른 하이퍼파라미터 (기본값 유지 또는 미세 조정)
            box=7.5,      # 박스 위치 정확도 가중치
            dfl=1.5,      # 분포 손실 가중치
            
            optimizer='AdamW', lr0=0.0005, lrf=0.01, cos_lr=True,
            mosaic=1.0, mixup=0.1, close_mosaic=10,
            
            device=0, workers=0, cache=False,
            name='tank_yolo_cbam_cls_tuning', # 결과 폴더 이름 변경
            exist_ok=True
        )
        
        print("\n🎉 Class Loss Tuning 실험 완료!")
        
    except Exception as e:
        print(f"\n🚨 학습 중 오류 발생: {e}")

✅ [시스템] 'CBAM_Universal' 등록 완료.

🚀 [실험] Class Loss Tuning (배경 오인식 억제 실험)
   ▶ runs/detect/tank_yolo_cbam_640_final/weights/best.pt 로드 후 학습 시작
Transferred 484/484 items from pretrained weights
Ultralytics 8.3.235  Python-3.10.19 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=2.0, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=C:/Users/hyun8/Desktop/Arakon/detection-base-6/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=800, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0

d:\anaconda\envs\yolo\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access  (ping: 0.10.0 ms, read: 121.128.9 MB/s, size: 51.8 KB)
val: Scanning C:\Users\hyun8\Desktop\Arakon\detection-base-6\valid\labels.cache... 132 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 132/132 131.9Kit/s 0.0s
Plotting labels to C:\Users\hyun8\Desktop\Arakon\runs\detect\tank_yolo_cbam_cls_tuning\labels.jpg... 
optimizer: AdamW(lr=0.0005, momentum=0.937) with parameter groups 77 weight(decay=0.0), 93 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 800 train, 800 val
Using 0 dataloader workers
Logging results to C:\Users\hyun8\Desktop\Arakon\runs\detect\tank_yolo_cbam_cls_tuning
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      1/100      2.57G     0.8456      4.028      

d:\anaconda\envs\yolo\lib\site-packages\torch\autograd\graph.py:825: UserWarning: adaptive_max_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\Context.cpp:95.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      1/100      2.79G      1.051       3.88      1.587         10        800: 100% ━━━━━━━━━━━━ 346/346 2.4it/s 2:22<0.3s2
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 17/17 1.2s/it 19.7s1.1s
                   all        132        271      0.836      0.798      0.877      0.603

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      2/100      3.25G     0.8336      3.784      1.395         13        800: 0% ──────────── 0/346  0.2s

...

     98/100      3.26G     0.5917      1.332      1.185         10        800: 100% ━━━━━━━━━━━━ 346/346 4.7it/s 1:14<0.2ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 17/17 1.9it/s 9.0s0.6s
                   all        132        271      0.937      0.865      0.922       0.69

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     99/100      3.26G     0.6506      1.468       1.43          5        800: 0% ──────────── 0/346  0.2s

d:\anaconda\envs\yolo\lib\site-packages\torch\autograd\graph.py:825: UserWarning: adaptive_max_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\Context.cpp:95.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


     99/100      3.26G     0.5902      1.383      1.177          8        800: 100% ━━━━━━━━━━━━ 346/346 4.6it/s 1:16<0.2ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 17/17 1.9it/s 9.0s0.6s
                   all        132        271      0.936       0.87      0.922      0.691

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
    100/100      3.26G      0.527       1.23      1.385          5        800: 0% ──────────── 0/346  0.2s

d:\anaconda\envs\yolo\lib\site-packages\torch\autograd\graph.py:825: UserWarning: adaptive_max_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\Context.cpp:95.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


    100/100      3.26G     0.5904      1.333      1.187          4        800: 100% ━━━━━━━━━━━━ 346/346 4.7it/s 1:14<0.2ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 17/17 1.8it/s 9.2s0.6s
                   all        132        271      0.957      0.847      0.923      0.694

100 epochs completed in 2.566 hours.
Optimizer stripped from C:\Users\hyun8\Desktop\Arakon\runs\detect\tank_yolo_cbam_cls_tuning\weights\last.pt, 52.2MB
Optimizer stripped from C:\Users\hyun8\Desktop\Arakon\runs\detect\tank_yolo_cbam_cls_tuning\weights\best.pt, 52.2MB

Validating C:\Users\hyun8\Desktop\Arakon\runs\detect\tank_yolo_cbam_cls_tuning\weights\best.pt...
Ultralytics 8.3.235  Python-3.10.19 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
YOLOv8m_cbam_real_final summary (fused): 116 layers, 25,906,303 parameters, 0 gradients, 78.8 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━